In [4]:
import pandas as pd
from pyproj import Transformer

In [6]:
# df = pd.read_csv("daejeon_local_hospital.csv", encoding='utf-8')
df = pd.read_csv("(도로명주소기준)_daejeon_local_hospital_transformed.csv", encoding='utf-8')
df.columns.to_list()

['번호',
 '개방서비스명',
 '개방서비스아이디',
 '개방자치단체코드',
 '관리번호',
 '인허가일자',
 '영업상태명',
 '상세영업상태코드',
 '상세영업상태명',
 '소재지전화',
 '소재지우편번호',
 '소재지전체주소',
 '도로명전체주소',
 '도로명우편번호',
 '사업장명',
 '최종수정시점',
 '데이터갱신구분',
 '데이터갱신일자',
 '업태구분명',
 '좌표정보x(epsg5174)',
 '좌표정보y(epsg5174)',
 '의료기관종별명',
 '의료인수',
 '입원실수',
 '병상수',
 '총면적',
 '진료과목내용',
 '진료과목내용명',
 '지정취소일자',
 '완화의료지정형태',
 '경도_X',
 '위도_Y']

In [7]:
df.head(3)

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,영업상태명,상세영업상태코드,상세영업상태명,소재지전화,...,의료인수,입원실수,병상수,총면적,진료과목내용,진료과목내용명,지정취소일자,완화의료지정형태,경도_X,위도_Y
0,1,병원,01_01_01_P,6300000,200463000002000000,2006-07-10,영업/정상,1,정상,424709000,...,75,57,160,13753.44,NaN,NaN,NaN,NaN,127.379533,36.346294
1,2,병원,01_01_01_P,6300000,201263000002000000,2012-06-08,영업/정상,1,정상,426347582,...,22,25,149,2941.47,NaN,NaN,NaN,NaN,127.446885,36.346257
2,3,병원,01_01_01_P,6300000,199763000002000000,1997-03-13,영업/정상,1,정상,422515105,...,25,5,7,4650.31,NaN,NaN,NaN,NaN,127.414021,36.337882


In [ ]:
x_point = df['좌표정보x(epsg5174)'].astype(float).to_list()  # 경도
y_point = df['좌표정보y(epsg5174)'].astype(float).to_list()  # 위도

# EPSG:5174는 국내 행정지도 측량용 좌표
# EPSG:4326은 전 세계 지도 서비스 표준 좌표
# 좌표 변환기 정의, epsg:5174 -> epsg:4326 # 변환 규칙 명시
transformer = Transformer.from_crs("epsg:5174", "epsg:4326", always_xy=True)

lon_list, lat_list = transformer.transform(x_point, y_point)

df['경도_X(epsg4326)'] = lon_list
df['위도_Y(epsg4326)'] = lat_list

# 기존 좌표계 삭제
df.drop(columns=['좌표정보x(epsg5174)', '좌표정보y(epsg5174)'], inplace=True)

# 변환된 데이터 확인
df[['사업장명', '위도_Y(epsg4326)', '경도_X(epsg4326)']].head(3) # 구글 대차비교용 출력

,사업장명,위도_Y(epsg4326),경도_X(epsg4326)
0,대전대학교 대전한방병원,36.346538,127.379329
1,의료법인 원전의료재단 가양요양병원,36.346322,127.446932
2,의료법인 영훈의료재단 선치과병원,36.337892,127.414018


# 대전대학교 대전한방병원	| 36.346538 | 127.379329
# https://www.google.com/maps?q=36.34653792451366,127.37932946833648
# 구글에 경우 위도(y), 경도(x) 순서로 입력.
# 네이버는 api 변수 입력 규칙 문서 참조 필요.

In [30]:
df.to_csv("(conversion)_daejeon_local_hospital_transformed.csv", index=False, encoding='utf-8')

In [32]:
saveing_check_df = pd.read_csv("(conversion)_daejeon_local_hospital_transformed.csv", encoding='utf-8')
saveing_check_df.columns.to_list()

['번호',
 '개방서비스명',
 '개방서비스아이디',
 '개방자치단체코드',
 '관리번호',
 '인허가일자',
 '영업상태명',
 '상세영업상태코드',
 '상세영업상태명',
 '소재지전화',
 '소재지우편번호',
 '소재지전체주소',
 '도로명전체주소',
 '도로명우편번호',
 '사업장명',
 '최종수정시점',
 '데이터갱신구분',
 '데이터갱신일자',
 '업태구분명',
 '의료기관종별명',
 '의료인수',
 '입원실수',
 '병상수',
 '총면적',
 '진료과목내용',
 '진료과목내용명',
 '지정취소일자',
 '완화의료지정형태',
 '경도_X(epsg4326)',
 '위도_Y(epsg4326)']

In [44]:
# 컨버전 저장 확인용 출력
saveing_check_df[['사업장명', '도로명전체주소', '위도_Y(epsg4326)', '경도_X(epsg4326)']].head(3)  

,사업장명,도로명전체주소,위도_Y(epsg4326),경도_X(epsg4326)
0,대전대학교 대전한방병원,"대전광역시 서구 대덕대로176번길 75, (둔산동)층 (둔산동)",36.346538,127.379329
1,의료법인 원전의료재단 가양요양병원,"대전광역시 동구 우암로 281, (가양동)층 (가양동)",36.346322,127.446932
2,의료법인 영훈의료재단 선치과병원,"대전광역시 중구 대종로 645-0, 부원빌딩층 (중촌동,부원빌딩)",36.337892,127.414018


In [49]:
#########################################################
# 상기 수학적 변환 함수는 오차 범위가 생각보다 크므로 
# 도로명 주소를 기반하여 재변환하는 작업 수행이 가능한지 확인하는 코드. 
import requests, json, time

KAKAO_API_KEY = "ab8b5c5d888399d743f09dd535be83a5"

url = "https://dapi.kakao.com/v2/local/search/address.json"
headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
params = {"query": "대전광역시 서구 대덕대로176번길 75, (둔산동)층 (둔산동)"}

res = requests.get(url, headers=headers, params=params)
res.json() # 데이터 응답 확인용

{'documents': [{'address': {'address_name': '대전 서구 둔산동 1136',
    'b_code': '3017011200',
    'h_code': '3017064000',
    'main_address_no': '1136',
    'mountain_yn': 'N',
    'region_1depth_name': '대전',
    'region_2depth_name': '서구',
    'region_3depth_h_name': '둔산2동',
    'region_3depth_name': '둔산동',
    'sub_address_no': '',
    'x': '127.379533037643',
    'y': '36.3462940178575'},
   'address_name': '대전 서구 대덕대로176번길 75',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '대전 서구 대덕대로176번길 75',
    'building_name': '대전대학교둔산캠퍼스',
    'main_building_no': '75',
    'region_1depth_name': '대전',
    'region_2depth_name': '서구',
    'region_3depth_name': '둔산동',
    'road_name': '대덕대로176번길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.379533037643',
    'y': '36.3462940178575',
    'zone_no': '35235'},
   'x': '127.379533037643',
   'y': '36.3462940178575'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [51]:
# 도로명 주소 기준하여 더 정확한 위도, 경도 변환 작업이 가능함을 확인하여. 
# 위도, 경도 변환 작업 함수 정의하고 반복문 통해 해당 함수 호출. 

def get_coords_from_address(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}
    
    res = requests.get(url, headers=headers, params=params)
    if res.status_code == 200:
        result = res.json()
        if result['documents']:
            x = result['documents'][0]['x']  # 경도
            y = result['documents'][0]['y']  # 위도
            return float(x), float(y)
        
    time.sleep(0.2)  # API 호출 제한에 걸리지 않도록 잠시 대기
    return None, None

In [53]:
df = pd.read_csv("daejeon_local_hospital.csv", encoding='utf-8') # 원본 데이터 재로딩
df[['경도_X', '위도_Y']] = df['도로명전체주소'].apply( # apply()는 python Pandas 라이브러리에서 반복을 대체하는 대표 함수
    lambda addr: pd.Series(get_coords_from_address(addr))
)
df[['사업장명', '도로명전체주소', '위도_Y', '경도_X']].head(3)  # 변환된 데이터 확인용 출력

,사업장명,도로명전체주소,위도_Y,경도_X
0,대전대학교 대전한방병원,"대전광역시 서구 대덕대로176번길 75, (둔산동)층 (둔산동)",36.346294,127.379533
1,의료법인 원전의료재단 가양요양병원,"대전광역시 동구 우암로 281, (가양동)층 (가양동)",36.346257,127.446885
2,의료법인 영훈의료재단 선치과병원,"대전광역시 중구 대종로 645-0, 부원빌딩층 (중촌동,부원빌딩)",36.337882,127.414021


In [54]:
df.to_csv("(도로명주소기준)_daejeon_local_hospital_transformed.csv", index=False, encoding='utf-8')
saveing_check_df = pd.read_csv("(도로명주소기준)_daejeon_local_hospital_transformed.csv", encoding='utf-8')
saveing_check_df[['사업장명', '도로명전체주소', '위도_Y', '경도_X']].head(3)  # 변환된 데이터 확인용 출력

,사업장명,도로명전체주소,위도_Y,경도_X
0,대전대학교 대전한방병원,"대전광역시 서구 대덕대로176번길 75, (둔산동)층 (둔산동)",36.346294,127.379533
1,의료법인 원전의료재단 가양요양병원,"대전광역시 동구 우암로 281, (가양동)층 (가양동)",36.346257,127.446885
2,의료법인 영훈의료재단 선치과병원,"대전광역시 중구 대종로 645-0, 부원빌딩층 (중촌동,부원빌딩)",36.337882,127.414021


In [ ]:
# 대전대학교 대전한방병원	| 
# 위도_Y : 36.346538 -> 36.346294 
# 경도_X : 127.379329 -> 127.379533

#  -> 수학적 변환 후
#   https://www.google.com/maps?q=36.34653792451366,127.37932946833648
#     -> 도로명 기준으로 변환 후
#       https://www.google.com/maps?q=36.346294,127.379533
#       https://google.com/maps?q=36.337882,127.414021